<a href="https://www.kaggle.com/code/karan842/500-species-classification-cnn-w-b-92?scriptVersionId=120101437" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 🦉🕊️ Bird Species Classification over 500 bird species

### Classifying 500 bird species using CNN architecutre

*In this notebook I am going to build CNN architecture for classifying 500 species of Birds*

- **We will perform some fundmental steps to build CNN model**:

   
   1. Importing libraries: Importing some necessary libraries which are needed for this project
   2. Setting [Weights & Biases](https://wandb.ai/home/) to track the experiemnts 
   3. Loading an image directory and converting them into dataframe with labels as a target species
   4. EDA: Analyzing the target classes and displaying sample set of images.
   5. Data Preprocessing: Performing Data Augmentation, resizing, rescaling 
   6. Building CNN model using Keras Pre-trained models
   7. Tracking the experiments using W&B
   8. Evaluating the model in a detail
  
 
 ## Notebook Extension:
 
 > I am going to create a end to end application either using FastAPI or Streamlit to deploy the model in real world environmet using MLOps. Click about GitHub link to know more

**[GitHub Link](https://github.com/karan842/bird-species-classification/)**

# Importing libraries

In [24]:
# Classic data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import PIL
from sklearn.model_selection import train_test_split

# tensorflow libaries
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Conv2D, BatchNormalization,MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model
from tensorflow.keras.applications import MobileNetV2, VGG19
from tensorflow.keras.layers.experimental import preprocessing

# System libraries
from pathlib import Path
import tqdm
import warnings
import glob
import os

# Metrics
from sklearn import metrics
import itertools

# WANDB 
import wandb
wandb.login()

# Noteboook setting
%matplotlib inline
sns.set_style('darkgrid')
warnings.filterwarnings("ignore")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


# Load and transform data

In [25]:
dataset = '/kaggle/input/100-bird-species/train/'

In [26]:
species_types=os.listdir(dataset)
# species_types

## Placing data into a DataFrame

In [27]:
image_dir = Path(dataset)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],filepaths))

# print(labels)

KeyboardInterrupt: 

In [ ]:
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels,name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels],axis=1)
image_df

# Exploratory Data Analyis

In [ ]:
image_df['Label'].value_counts(ascending=False)

- Count of each class is ranging between 248 to 130

## Top 100 most occured Bird Species

In [ ]:
class_freq = {}
class_labels = image_df['Label']
for label in class_labels:
    if label in class_freq:
        class_freq[label] +=1
    else:
        class_freq[label] = 1
        
# sort the classes by frequency in descending order
sorted_class = sorted(class_freq.items(),key=lambda x: x[1],reverse=True)
# print(sorted_class)
top_classes = dict(sorted_class[:20])

plt.figure(figsize=(10,8))
sns.countplot(y=class_labels, order=top_classes.keys())
plt.xlabel("Frequency")
plt.ylabel("Class Label")
plt.title("Top 10 classes with most occurrences")
plt.show();

## Different types of common bird species such as Eagle, Chicken, Pigeon, Raven, Vulture, Sparrow, Owl and Duck

In [ ]:
# Function to fing different types of spcies of some common birds
def diff_species_common(specie,df):
    bird_species = df[df['Label'].str.contains(specie,case=False)]['Label']
    bird_species = bird_species.unique()
    return bird_species

print("Different types of species for an Eagle: \n")
print(diff_species_common('EAGLE', image_df))
print("\n\nDifferent types of species for a Pigeon: \n")
print(diff_species_common('PIGEON',image_df))
print("\n\nDifferent types of species for a Chicken: \n")
print(diff_species_common('CHICKEN',image_df))
print("\n\nDifferent types of species for a Duck: \n")
print(diff_species_common('DUCK',image_df))
print("\n\nDifferent types of species for a Vulture: \n")
print(diff_species_common('SPARROW',image_df))
print("\n\nDifferent types of species for an Owl: \n")
print(diff_species_common('OWL',image_df))
print("\n\nDifferent types of species for a Raven: \n")
print(diff_species_common('RAVEN',image_df))
print("\n\nDifferent types of species for a Sparrow: \n")
print(diff_species_common('SPARROW',image_df))

## Visualizing sample images from dataset

In [ ]:
random_index = np.random.randint(0,len(image_df),25)
fig, axes = plt.subplots(nrows=5,ncols=5,figsize=(10,10),
                        subplot_kw={'xticks':[],'yticks':[]})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show();

## Data Preprocssing
The data will be split into three different categories: Training, Validation and Testing. The training data will be used to train the deep learning CNN model and its parameters will be fine tuned with the validation data. Finally, the performance of the data will be evaluated using the test data(data the model has not previously seen).

In [ ]:
# Seperate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2,
                                     shuffle=True,random_state=42)

# Hyperparameter tuning using W&B

## Initializing W&B

### Data Augmentation 

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range = 15,
    width_shift_range = 0.05,
    height_shift_range = 0.05,
    rescale = 1./255,
    shear_range = 0.05,
    brightness_range = [0.1,1.5],
    horizontal_flip = True,
    vertical_flip = True
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_images = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = val_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224,224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = val_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
# Resize Layer
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(224,224),
    layers.experimental.preprocessing.Rescaling(1./255),
])

## Training the model

In [ ]:
NUM_CLASSES = 500
base_model = MobileNetV2(
    input_shape=(224,224,3),
    include_top=False,
    weights='imagenet',
    pooling='avg',
    classes = NUM_CLASSES
)

for layer in base_model.layers:
        layer.trainable = False

In [ ]:
inputs = base_model.input
x = resize_and_rescale(inputs)
x = Dense(256, activation='relu')(base_model.output)
x = Dropout(0.8)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.8)(x)

outputs = Dense(500,activation='softmax')(x)

model =  Model(inputs=inputs, outputs=outputs)

model.summary()

## Weights and Biases configuration

In [ ]:
# Run 
from wandb.keras import WandbCallback
run = wandb.init(project='birds-species-classification',
                config={ # include hyperparameters and metadata
                    "learning_rate":0.0001,
                    "epochs":100,
                    "batch_size":32,
                    "es_patience":8,
                    "loss_function":"categorical_crossentropy",
                    
                })
config = wandb.config # we will use this to configure our experiment
tf.keras.backend.clear_session()

# model callbacks, optimizers and compilation

## Optimizer
optimizer = tf.keras.optimizers.Adam(config.learning_rate)

## Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=config.es_patience,
                              restore_best_weights=True)
# Create checkpoint callback
checkpoint_path = "birds_classification_model_checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Model compile

model.compile(optimizer,config.loss_function,metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_images, 
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=config.epochs,
    callbacks=[
        early_stopping,
        WandbCallback(),
        checkpoint_callback,
    ]
)

# Model Evaluation

In [ ]:
results = model.evaluate(test_images)
print("    Test Loss: {:.5f}%".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1]*100))

In [ ]:
def plot_loss_accuracy_curves(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show();
    
plot_loss_accuracy_curves(history)

## Making predictiong on the test data

Predict the label of test_images

In [ ]:
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)
pred

In [ ]:
# train_images.class_indices

Map the label

In [ ]:
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Result
print(f'First 5 predictions: {pred[:5]}')

### Display 25 Random images from dataset with their labels and predict with the model

In [ ]:
random_index = np.random.randint(0,len(test_df)-1,25)
fig, axes = plt.subplots(nrows=5, ncols=5,figsize=(25,15),
                        subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_index[i]]))
    if test_df.Label.iloc[random_index[i]] == pred[random_index[i]]:
        color = "green"
    else:
        color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}",
                color=color)

plt.show()
plt.tight_layout()

## Classification Report
We will convert our classification report into dataframe and find the best and worst classes using pandas operations for ease.

In [ ]:
species = test_images.class_indices

In [ ]:
y_test = list(test_df.Label)
report = metrics.classification_report(y_test, pred,target_names=species,output_dict=True)

data = []
for k,v in report.items():
    if k in species:
        data.append({'species':k,
                    'precision': v['precision'],
                    'recall': v['recall'],
                    'f1-score': v['f1-score'],
                    'support': v['support']})
df = pd.DataFrame(data)
df

# Above code is under development :)

### Lets see which TOP-10 classes has good and bad Precision/Recall/F1-Scores
F1-Score is a harmonic mean between **Precision and Recall**(trade-off). So, we will classes based on f1-score   

In [ ]:
def best_classes(df):
    print("Classed with good result: ")
    top_10_good = df.nlargest(10,'f1-score')
    res = top_10_good[['species','precision','recall','f1-score','support']]
    return res
    
best_classes(df)

In [ ]:
def worst_classes(df):
    print("Classed with bad result: ")
    top_10_bad = df.nsmallest(10,'f1-score')
    res = top_10_bad[['species','precision','recall','f1-score','support']]
    return res
    
worst_classes(df)

## Predict the bird

In [ ]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.applications.mobilenet_v2 import decode_predictions, preprocess_input



def predict_bird_species(model,img_path):
    
    # load the image
    img = load_img(img_path,target_size=(224,224))
    
    # convert the image to numpy array
    img_array = img_to_array(img)
    
    # preprocess the image for MobileNet model
    img_array = preprocess_input(img_array)
    
    # Make a prediction on the image
    predictions = model.predict(np.array([img_array]))
    
    # Decode the prediction results
    top_preds = decode_predictions(predictions, top=1)
    
    # Return the predicted class name
    return top_preds[0][0][1]
    

sample_img = '/kaggle/input/100-bird-species/valid/AZURE TIT/2.jpg'
predict_bird_species(model,sample_img)

## We will see the top-k accuracy of classes

Evaluating a classification model based on top-k accuracy is a useful approach when dealing with a large number of classes. Top-k accuracy measures the percentage of times that the correct label is among the top-k predicted labels. This is particularly useful for large classification problems, where the correct label may not be the highest predicted class.

> Note: Note that the top-k accuracy will typically be lower than the overall accuracy of the model, but it provides a more realistic evaluation metric for problems with a large number of output classes.

In [ ]:
y_true = [0, 1, 2, 3, 4]
y_pred = [5,2,7,1,3]

y_pred = np.array(y_pred)
print(y_pred.argsort()[:])

In [ ]:
pred

In [ ]:
# Define a function to calculate top-k accuracy
def top_k_accuracy(predictions, true_labels, k):
   # Convert predictions to a NumPy array
    predictions = np.array(predictions)
    
    # Sort the predictions by decreasing order of probability
    sorted_preds = np.argsort(predictions, axis=0)[::-1]
    
    # Calculate the number of correct predictions in the top-k
    num_correct = 0
    for i in range(len(true_labels)):
        if true_labels[i] in sorted_preds[i, :k]:
            num_correct += 1
    
    # Calculate the top-k accuracy
    accuracy = num_correct / len(true_labels)
    
    return accuracy

# Evaluate the model based on top-k accuracy
k = 3  # top-k value
accuracy = top_k_accuracy(pred, y_test, k)
print(f"Top-{k} Accuracy: {accuracy:.3f}")

In [ ]:
k=10
top_k_accuracy = metrics.top_k_accuracy_score(y_test,pred,k=k)
print(f"Top-{k} Accuracy: {top_k_accuracy:.3f}")

Exception in thread NetStatThr:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_run.py", line 152, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface.py", line 138, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 405, in _communicate_network_status
    resp = self._communicate(req, local=True)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/interface/interface_shared.py", line 226, in _communicate
    return self._communicate_async(rec, local=local).get(timeout=timeout)
  File "/opt/cond